### Finetuning OpenAI Model Exercise

We'll finetune an OpenAI model with custom data to create a custom agent

Inspiration taken from [here](https://www.youtube.com/watch?v=boHXgQ5eQic)

In [3]:
from datasets import load_dataset
import huggingface_hub

List of datasets ready for loading from the [HuggingfaceHub](https://huggingface.co/datasets)

In [8]:

[i for i in huggingface_hub.list_datasets()]

[DatasetInfo(id='acronym_identification', author=None, sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=992, likes=17, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'arxiv:2010.14678', 'region:us'], card_data=None, siblings=None),
 DatasetInfo(id='ade_corpus_v2', author=None, sha='4ba01c71687dd7c996597042449448ea312126cf', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 42, 58, tzinfo=datetime.timezone.utc), private=False, gated=Fa

We'll be using the following example [train dataset](https://huggingface.co/datasets/jamescalam/agent-conversations-retrieval-tool). This training messages were generated by GPT-4

**Note:** One can build their own dataset, it is not difficult to implement 

In [10]:
new_data = load_dataset(
    "jamescalam/agent-conversations-retrieval-tool",
    split="train" # which split to return
)

Generating train split: 270 examples [00:00, 41089.30 examples/s]


The training data has 270 messages, each containg a list of roles and content

**Each message represents a single conversation**. Note that all contain the same system message (which we can modify for our particular chatbot we want to train)

In [11]:
new_data

Dataset({
    features: ['messages'],
    num_rows: 270
})

For example, the first message has a set of exchanges between the user and assistant **roles** with **content** examples of how the assistant should behave when prompted with a type of content

In [17]:
print(type(new_data[0]))
new_data[0]

<class 'dict'>


{'messages': [{'role': 'system',
   'content': 'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a p

In [25]:
print(new_data[0]["messages"][1])

{'role': 'user', 'content': 'TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Vector Search Tool: This tool allows you to get research information about LLMs.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{\n    "action": string, \\ The action to take. Must be one of Vector Search Tool\n    "action_input": string \\ The input to the action\n}\n```\n\n**Option #2:**\nUse this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:\n\n```json\n{\n    "action": "Final Answer",\n    "action_input": string \\ You should put what you want to return to use here\n}\n```\n\nUSER\'S INPUT\n-------------

In [24]:
print(new_data[0]["messages"][1]["content"])

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Vector Search Tool: This tool allows you to get research information about LLMs.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{
    "action": string, \ The action to take. Must be one of Vector Search Tool
    "action_input": string \ The input to the action
}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{
    "action": "Final Answer",
    "action_input": string \ You should put what you want to return to use here
}
```

USER'S INPUT
--------------------
Here is the user's input (remember to respond with a markd

Now we convert the training data to json1 format

**Note on JSON1 extension:** One way to store JSON for SQLite: [link](https://www.beekeeperstudio.io/blog/sqlite-json)

In [26]:
new_data.to_json("TRAINING_DATA/jc_conversations_train.json1")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 72.81ba/s]


1103809

### Now that we explored and stored the training data, let's go ahead and train base gpt-3.5

**Note:** If you want to check the validity of the data for training you created, you can do so as stated [here](https://cookbook.openai.com/examples/chat_finetuning_data_prep)

Check out OpenAI's [finetuning training material](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

In [30]:
import os
from openai import OpenAI

In [35]:
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()

We upload our data for training to OpenAI

In [36]:
file_load = client.files.create(
    file=open("TRAINING_DATA/jc_conversations_train.json1", "rb"),
    purpose="fine-tune"
)

In [43]:
file_load

FileObject(id='file-RrC6cpvDUBdq8PA41EumgsG0', bytes=1103809, created_at=1705791884, filename='jc_conversations_train.json1', object='file', purpose='fine-tune', status='processed', status_details=None)

In [50]:
file_load.dict()

{'id': 'file-RrC6cpvDUBdq8PA41EumgsG0',
 'bytes': 1103809,
 'created_at': 1705791884,
 'filename': 'jc_conversations_train.json1',
 'object': 'file',
 'purpose': 'fine-tune',
 'status': 'processed',
 'status_details': None}

We take the file ID and create the finetuning job. We'll **finetune base gpt-3.5-turbo** in this example.

This might take some time to train since it might behind a queue

In [51]:
model_finetune = client.fine_tuning.jobs.create(training_file=file_load.dict()["id"],
                                                model="gpt-3.5-turbo",
                                                suffix="012024_jc_sample")

In [55]:
model_finetune.dict()

{'id': 'ftjob-nUFfwPtcCy6rMJ5ONgi99i4n',
 'created_at': 1705793231,
 'error': None,
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'n_epochs': 'auto',
  'batch_size': 'auto',
  'learning_rate_multiplier': 'auto'},
 'model': 'gpt-3.5-turbo-0613',
 'object': 'fine_tuning.job',
 'organization_id': 'org-HCpyLwy1gf2wzE49sCGQRU79',
 'result_files': [],
 'status': 'validating_files',
 'trained_tokens': None,
 'training_file': 'file-RrC6cpvDUBdq8PA41EumgsG0',
 'validation_file': None}